# **W1 Homework**

In [3]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

## **Read Data**

In [47]:
january = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')
february = pd.read_parquet('../data/yellow_tripdata_2022-02.parquet')

## **A first view of data**

In [11]:
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


## **Questions**

### **Q1. Downloading the data**
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

In [12]:
len(list(january.columns))

19

### **Q2. Computing duration**
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [13]:
january['duration'] = january.tpep_dropoff_datetime - january.tpep_pickup_datetime
january.duration = january.duration.apply(lambda td: td.total_seconds() / 60)

In [15]:
january['duration'].describe().apply(lambda x: format(x, 'f'))

count    2463931.000000
mean          14.212203
std           46.445305
min        -3442.400000
25%            6.316667
50%           10.183333
75%           16.166667
max         8513.183333
Name: duration, dtype: object

### **Q3. Dropping outliers**
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [39]:
january_without_outliers = january[(january['duration'] >=1) & (january['duration'] <=60)]

In [40]:
total_reamin_data = (len(january_without_outliers)/len(january))*100
print(f"Total data remain after removing outliers: {total_reamin_data}%")

Total data remain after removing outliers: 98.27547930522405%


### **Q4. One-hot encoding**
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

january_without_outliers[categorical] = january_without_outliers[categorical].astype(str)

In [42]:
train_dicts = january_without_outliers[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [43]:
target = 'duration'
y_train = january_without_outliers[target].values

In [44]:
X_train.shape

(2421440, 516)

### **Q5. Training a model**
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [46]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179360711

### **Q6. Evaluating the model**
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

In [52]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [53]:
df_train = read_dataframe('../data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('../data/yellow_tripdata_2022-02.parquet')

In [59]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [61]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [64]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179360711

In [65]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.795498400456978